# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [2]:
# Load API Credentials
with open('/Users/jacobwang/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

### Define Search Terms and File Paths

In [7]:
# set our API call parameters and filename before the first call
location = 'Philadelphia, PA'
term = 'italian'

In [5]:
## Specify fodler for saving data
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok=True)


In [8]:
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [9]:
JSON_FILE

'Data/Philadelphia-italian.json'

### Check if Json File exists and Create it if it doesn't

In [12]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(FOLDER, exist_ok=True)

        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w')as f:
        json.dump([], f)
        
## If it exists, inform user
else: 
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Philadelphia-italian.json not found. Saving empty list to file.


### Load JSON FIle and account for previous results

In [ ]:
## Load previous results and use len of results for offset

## set offset based on previous results


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [16]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=location,
                                       term=term)
print((results))
type(results)

{'businesses': [{'id': '-cEFKAznWmI0cledNOIQ7w', 'alias': 'gran-caffe-l-aquila-philadelphia', 'name': "Gran Caffe L'Aquila", 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/xKGtEMHg8-r_8XyaC585DQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/gran-caffe-l-aquila-philadelphia?adjust_creative=EPlkYaSpsl5pSK06lCD3BQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=EPlkYaSpsl5pSK06lCD3BQ', 'review_count': 1290, 'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'}, {'alias': 'italian', 'title': 'Italian'}, {'alias': 'gelato', 'title': 'Gelato'}], 'rating': 4.5, 'coordinates': {'latitude': 39.95127, 'longitude': -75.1695}, 'transactions': ['delivery', 'pickup'], 'price': '$$', 'location': {'address1': '1716 Chestnut St', 'address2': '', 'address3': '', 'city': 'Philadelphia', 'zip_code': '19103', 'country': 'US', 'state': 'PA', 'display_address': ['1716 Chestnut St', 'Philadelphia, PA 19103']}, 'phone': '+12155685600', 'display_phone': '(215) 5

dict

In [17]:
## How many results total?
len(results)

3

- Where is the actual data we want to save?

In [18]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [19]:
results['businesses'][0]

{'id': '-cEFKAznWmI0cledNOIQ7w',
 'alias': 'gran-caffe-l-aquila-philadelphia',
 'name': "Gran Caffe L'Aquila",
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/xKGtEMHg8-r_8XyaC585DQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/gran-caffe-l-aquila-philadelphia?adjust_creative=EPlkYaSpsl5pSK06lCD3BQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=EPlkYaSpsl5pSK06lCD3BQ',
 'review_count': 1290,
 'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'},
  {'alias': 'italian', 'title': 'Italian'},
  {'alias': 'gelato', 'title': 'Gelato'}],
 'rating': 4.5,
 'coordinates': {'latitude': 39.95127, 'longitude': -75.1695},
 'transactions': ['delivery', 'pickup'],
 'price': '$$',
 'location': {'address1': '1716 Chestnut St',
  'address2': '',
  'address3': '',
  'city': 'Philadelphia',
  'zip_code': '19103',
  'country': 'US',
  'state': 'PA',
  'display_address': ['1716 Chestnut St', 'Philadelphia, PA 19103']},
 'phone': '+12155685600',
 'display_

In [20]:
results['total']

3200

In [21]:
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,-cEFKAznWmI0cledNOIQ7w,gran-caffe-l-aquila-philadelphia,Gran Caffe L'Aquila,https://s3-media3.fl.yelpcdn.com/bphoto/xKGtEM...,False,https://www.yelp.com/biz/gran-caffe-l-aquila-p...,1290,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 39.95127, 'longitude': -75.1695}","[delivery, pickup]",$$,"{'address1': '1716 Chestnut St', 'address2': '...",+12155685600,(215) 568-5600,313.234911
1,6ajnOk0GcY9xbb5Ocaw8Gw,barbuzzo-philadelphia,Barbuzzo,https://s3-media3.fl.yelpcdn.com/bphoto/vCTPQ2...,False,https://www.yelp.com/biz/barbuzzo-philadelphia...,3081,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.5,"{'latitude': 39.94999, 'longitude': -75.16216}","[delivery, pickup]",$$,"{'address1': '110 S 13th St', 'address2': '', ...",+12155469300,(215) 546-9300,877.931790
2,lKU_kljhbEQKIPaM1Zbqiw,little-nonnas-philadelphia,Little Nonna's,https://s3-media2.fl.yelpcdn.com/bphoto/0dQJ4d...,False,https://www.yelp.com/biz/little-nonnas-philade...,1251,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 39.9477864380196, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '1234 Locust St', 'address2': '',...",+12155462100,(215) 546-2100,1036.521920
3,dYinIkKBspHV5hSaukklFg,l-angolo-ristorante-philadelphia-3,L'Angolo Ristorante,https://s3-media1.fl.yelpcdn.com/bphoto/_-xeA0...,False,https://www.yelp.com/biz/l-angolo-ristorante-p...,409,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 39.9196243286133, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '1415 W Porter St', 'address2': N...",+12153894252,(215) 389-4252,3797.786432
4,-bV-HbEV9F0qXvmukmQwbw,luna-byob-philadelphia,Luna BYOB,https://s3-media4.fl.yelpcdn.com/bphoto/kR-O3p...,False,https://www.yelp.com/biz/luna-byob-philadelphi...,6,"[{'alias': 'italian', 'title': 'Italian'}]",5.0,"{'latitude': 39.949678356249585, 'longitude': ...",[],NaN,"{'address1': '227 S 20th St', 'address2': '', ...",+12156932220,(215) 693-2220,507.314834
5,eLbKuFpCtHEjCkoWkDDk7g,trattoria-carina-philadelphia-2,Trattoria Carina,https://s3-media4.fl.yelpcdn.com/bphoto/Q3N-1y...,False,https://www.yelp.com/biz/trattoria-carina-phil...,280,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 39.94878, 'longitude': -75.17803}",[delivery],$$,"{'address1': '2201 Spruce St', 'address2': '',...",+12157325818,(215) 732-5818,798.325153
6,qnprJxrjwuEivitA8V2aEQ,fiorella-philadelphia-2,Fiorella,https://s3-media3.fl.yelpcdn.com/bphoto/ntu7O8...,False,https://www.yelp.com/biz/fiorella-philadelphia...,205,"[{'alias': 'pastashops', 'title': 'Pasta Shops...",4.5,"{'latitude': 39.9388677767588, 'longitude': -7...",[delivery],$$$,"{'address1': '817 Christian St', 'address2': '...",+12153059222,(215) 305-9222,2056.445857
7,xdc0TbKkHMZN3N82oeq2XA,via-locusta-philadelphia,Via Locusta,https://s3-media3.fl.yelpcdn.com/bphoto/B9sNFi...,False,https://www.yelp.com/biz/via-locusta-philadelp...,230,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 39.9490553, 'longitude': -75.1704...",[delivery],$$$,"{'address1': '1723 Locust St', 'address2': '',...",+12156420020,(215) 642-0020,540.974464
8,sVZb87xjhbCWHQ2UXOGLrA,giorgio-on-pine-philadelphia,Giorgio On Pine,https://s3-media1.fl.yelpcdn.com/bphoto/EwboUh...,False,https://www.yelp.com/biz/giorgio-on-pine-phila...,1040,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 39.945339381185, 'longitude': -75...","[delivery, pickup]",$$,"{'address1': '1328 Pine St', 'address2': '', '...",+12155456265,(215) 545-6265,1101.049702
9,2qVZiMB22YPC2HCM62Gnbw,scarpetta-philadelphia,Scarpetta,https://s3-media2.fl.yelpcdn.com/bphoto/G5lGck...,False,https://www.yelp.com/biz/scarpetta-philadelphi...,235,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 39.9498657276508, 'longitude': -7...","[delivery, pickup]",$$$,"{'address1': '210 W Rittenhouse Squa

In [22]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [25]:
# Use math.ceil to round up for the total number of pages of results.
import time, math
n_pages = math.ceil((results['total'])/results_per_page)
n_pages

160

In [26]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
        try:
            
            time.sleep(.2)
        
        ## Read in results in progress file and check the length
            with open(JSON_FILE, 'r') as f:
                previous_results = json.load(f)
        
        ## save number of results for to use as offset
            n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
            results = yelp_api.search_query(location = location, 
                                           term = term,
                                           offset = n_results+1)

        ## append new results and save to file
            previous_results.extend(results['businesses'])
            with open(JSON_FILE, 'w') as f:
                json.dump(previous_results, f)
            
    ## What to do if we get an error/exception.
        except Exception as e: 
            print('[!] ERROR: ', e)


  0%|          | 0/160 [00:00<?, ?it/s]

[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: To

[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: To

## Open the Final JSON File with Pandas

In [27]:
df = pd.read_json(JSON_FILE)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,6ajnOk0GcY9xbb5Ocaw8Gw,barbuzzo-philadelphia,Barbuzzo,https://s3-media3.fl.yelpcdn.com/bphoto/vCTPQ2...,False,https://www.yelp.com/biz/barbuzzo-philadelphia...,3081,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.5,"{'latitude': 39.94999, 'longitude': -75.16216}","[pickup, delivery]",$$,"{'address1': '110 S 13th St', 'address2': '', ...",+12155469300,(215) 546-9300,877.931790
1,lKU_kljhbEQKIPaM1Zbqiw,little-nonnas-philadelphia,Little Nonna's,https://s3-media2.fl.yelpcdn.com/bphoto/0dQJ4d...,False,https://www.yelp.com/biz/little-nonnas-philade...,1251,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 39.9477864380196, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '1234 Locust St', 'address2': '',...",+12155462100,(215) 546-2100,1036.521920
2,dYinIkKBspHV5hSaukklFg,l-angolo-ristorante-philadelphia-3,L'Angolo Ristorante,https://s3-media1.fl.yelpcdn.com/bphoto/_-xeA0...,False,https://www.yelp.com/biz/l-angolo-ristorante-p...,409,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 39.9196243286133, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '1415 W Porter St', 'address2': N...",+12153894252,(215) 389-4252,3797.786432
3,-bV-HbEV9F0qXvmukmQwbw,luna-byob-philadelphia,Luna BYOB,https://s3-media4.fl.yelpcdn.com/bphoto/kR-O3p...,False,https://www.yelp.com/biz/luna-byob-philadelphi...,6,"[{'alias': 'italian', 'title': 'Italian'}]",5.0,"{'latitude': 39.949678356249585, 'longitude': ...",[],NaN,"{'address1': '227 S 20th St', 'address2': '', ...",+12156932220,(215) 693-2220,507.314834
4,eLbKuFpCtHEjCkoWkDDk7g,trattoria-carina-philadelphia-2,Trattoria Carina,https://s3-media4.fl.yelpcdn.com/bphoto/Q3N-1y...,False,https://www.yelp.com/biz/trattoria-carina-phil...,280,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 39.94878, 'longitude': -75.17803}",[delivery],$$,"{'address1': '2201 Spruce St', 'address2': '',...",+12157325818,(215) 732-5818,798.325153


In [28]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Philadelphia-italian.csv.gz'

In [29]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression = 'gzip', index = False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [30]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 966,796 Bytes
CSV.GZ FILE: 146,438 Bytes
the csv.gz is 6.6020841584834535 times smaller!


## Next Class: Processing the Results and Mapping 